In [1]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os,random, math
from tqdm import tqdm
from copy import deepcopy
from collections import Counter

# Visualization
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import font_manager, rc
plt.rcParams['font.family'] = 'NanumGothic'
import platform
if platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
    rc('font', family=font_name)
else:    
    rc('font', family='AppleGothic')

matplotlib.rcParams['axes.unicode_minus'] = False

# from dataprep.eda import plot, plot_correlation, plot_missing

import plotly 
import plotly.express as px
# from plotly import tools, subplots
# import plotly.offline as py
# py.init_notebook_mode(connected=True)
# from plotly.offline import init_notebook_mode, iplot
# import plotly.graph_objs as go
# import plotly.express as px

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [4]:
import os
os.chdir("/Users/seungji/Desktop/Dacon/Daesamanlap")

**행정동 법정동 매핑**
- 출처: 행정안전부
- 링크: https://www.mois.go.kr/frt/bbs/type001/commonSelectBoardArticle.do;jsessionid=+ZUzK-vRbyaKqumcJq8NrYJf.node50?bbsId=BBSMSTR_000000000052&nttId=85215

In [3]:
mapping = pd.read_excel("법정동행정동매핑.xlsx")
mapping

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
0,1100000000,서울특별시,NaN,NaN,1100000000,서울특별시,19880423,nan
1,1111000000,서울특별시,종로구,NaN,1111000000,종로구,19880423,nan
2,1111051500,서울특별시,종로구,청운효자동,1111010100,청운동,20081101,nan
3,1111051500,서울특별시,종로구,청운효자동,1111010200,신교동,20081101,nan
4,1111051500,서울특별시,종로구,청운효자동,1111010300,궁정동,20081101,nan
...,...,...,...,...,...,...,...,...
21734,5013061000,제주특별자치도,서귀포시,중문동,5013011400,대포동,20060701,nan
21735,5013061000,제주특별자치도,서귀포시,중문동,5013011800,하원동,20060701,nan
21736,5013062000,제주특별자치도,서귀포시,예래동,5013011900,색달동,20060701,nan
21737,5013062000,제주특별자치도,서귀포시,예래동,5013012000,상예동,20060701,nan


#### 건축연면적

In [94]:
bld_tr = pd.read_csv("External3/building_tr.csv")
bld_tst = pd.read_csv("External3/building_tst.csv")

#### 법정동별 인구수 및 세대수 및 ... 

- 링크: https://kostat.go.kr/portal/korea/index.action

In [6]:
pop1 = pd.read_excel("/Users/seungji/Desktop/Dacon/Daesamanlap/External2/202106주민등록인구및세대현황_월간.xlsx")
pop2 = pd.read_excel("/Users/seungji/Desktop/Dacon/Daesamanlap/External2/202106연령별인구현황_월간.xlsx")

- 건축연면적의 지번주소는 법정동 기준으로 되어있다. 
- 법정동별 인구정보는 행정동 기준으로 되어있다. 
- 이 두 데이터를 잇기 위해 행정동/법정동 매핑 데이터가 필요하다. 

#### 순서
1. `mapping`에 `pop1`과 `pop2`를 merge -> `mapping1`
    - key: mapping의 `행정동코드`와 pop1과 pop2의 `행정기관코드`를 기준으로
    - 
1. `건축연면적`에 `mapping1`를 merge
    - key: 

## 1. mapping에 pop1과 pop2를 merge -> mapping2

In [11]:
len(set(mapping.행정동코드)& set(pop1.행정기관코드))

3835

In [20]:
len(set(mapping.행정동코드)& set(pop2.행정기관코드))

3837

In [24]:
pop1[pop1.행정기관코드.isin(list(set(pop2.행정기관코드)-set(mapping.행정동코드)))]

,행정기관코드,행정기관,총인구수,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율
434,1174052000,서울특별시 강동구 상일동,"38,683","13,380",2.8900,"19,080","19,603",0.9700
833,2811060000,인천광역시 중구 북성동,"2,744","1,507",1.8200,"1,423","1,321",1.0800
834,2811061000,인천광역시 중구 송월동,"4,378","2,153",2.0300,"2,157","2,221",0.9700
1615,4139059500,경기도 시흥시 배곧동,"71,154","27,314",2.6100,"36,445","34,709",1.0500
1704,4148038700,경기도 파주시 장단출장소,552,216,2.5600,278,274,1.0100
1705,4148040000,경기도 파주시 진동면,164,74,2.2200,90,74,1.2200
1789,4159400000,경기도 화성시동탄출장소,0,0,0.0000,0,0,0.0000
3209,4719052000,경상북도 구미시 원평1동,"3,716","2,711",1.3700,"2,074","1,642",1.2600
3210,4719053000,경상북도 구미시 원평2동,"2,757","1,668",1.6500,"1,443","1,314",1.1000
3473,4812155000,경상남도 창원시 의창구 용지동,"29,735","13,242",2.2500,"14,933","14,802",1.0100


In [ ]:
{"서울특별시 강동구 상일동":1174052500,}

In [27]:
mapping[(mapping.시도명 == "인천광역시")&(mapping.시군구명 == "중구")]

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자
1549,2811000000,인천광역시,중구,NaN,2811000000,중구,19950101,nan
1550,2811052000,인천광역시,중구,연안동,2811011800,항동7가,19950101,nan
1551,2811052000,인천광역시,중구,연안동,2811013800,북성동1가,19950101,nan
1552,2811053000,인천광역시,중구,신포동,2811010100,중앙동1가,19981010,nan
1553,2811053000,인천광역시,중구,신포동,2811010200,중앙동2가,19981010,nan
1554,2811053000,인천광역시,중구,신포동,2811010300,중앙동3가,19981010,nan
1555,2811053000,인천광역시,중구,신포동,2811010400,중앙동4가,19981010,nan
1556,2811053000,인천광역시,중구,신포동,2811010500,해안동1가,19981010,nan
1557,2811053000,인천광역시,중구,신포동,2811010600,해안동2가,19981010,nan
1558,2811053000,인천광역시,중구,신포동,2811010700,해안동3가,19981010,nan


In [18]:
pop1.행정기관코드.value_counts().value_counts()


1    3846
Name: 행정기관코드, dtype: int64

In [19]:
pop2.행정기관코드.value_counts().value_counts()


1    3847
Name: 행정기관코드, dtype: int64

In [30]:
print(mapping.shape)

(21739, 8)


In [29]:
mapping1 = mapping.merge(pop1,left_on = "행정동코드",right_on = "행정기관코드",how = "left")

In [31]:
print(mapping1.shape)

(21739, 16)


In [33]:
mapping1 = mapping1.merge(pop2,left_on = "행정동코드",right_on = "행정기관코드",how = "left")

In [34]:
print(mapping1.shape)

(21739, 57)


In [38]:
mapping1.columns

Index(['행정동코드', '시도명', '시군구명', '읍면동명', '법정동코드', '동리명', '생성일자', '말소일자',
       '행정기관코드_x', '행정기관_x', '총인구수_x', '세대수', '세대당 인구', '남자 인구수', '여자 인구수',
       '남여 비율', '행정기관코드_y', '행정기관_y', '총인구수_y', '총연령구간인구수', '총0~9세', '총10~19세',
       '총20~29세', '총30~39세', '총40~49세', '총50~59세', '총60~69세', '총70~79세',
       '총80~89세', '총90~99세', '총100세이상', '남 인구수', '남연령구간인구수', '남0~9세',
       '남10~19세', '남20~29세', '남30~39세', '남40~49세', '남50~59세', '남60~69세',
       '남70~79세', '남80~89세', '남90~99세', '남100세 이상', '여 인구수', '여연령구간인구수',
       '여0~9세', '여10~19세', '여20~29세', '여30~39세', '여40~49세', '여50~59세',
       '여60~69세', '여70~79세', '여80~89세', '여90~99세', '여100세 이상'],
      dtype='object')

In [43]:
print(sum(mapping1.행정기관코드_x != mapping1.행정기관코드_y))
print(sum(mapping1.총인구수_x != mapping1.총인구수_y))
print(sum(mapping1.행정기관_x != mapping1.행정기관_y))

84
84
84


In [44]:
mapping1[mapping1.행정기관코드_x != mapping1.행정기관코드_y]

,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자,행정기관코드_x,행정기관_x,총인구수_x,세대수,세대당 인구,남자 인구수,여자 인구수,남여 비율,행정기관코드_y,행정기관_y,총인구수_y,총연령구간인구수,...,남40~49세,남50~59세,남60~69세,남70~79세,남80~89세,남90~99세,남100세 이상,여 인구수,여연령구간인구수,여0~9세,여10~19세,여20~29세,여30~39세,여40~49세,여50~59세,여60~69세,여70~79세,여80~89세,여90~99세,여100세 이상
751,1174052500,서울특별시,강동구,상일제1동,1174010300,상일동,20210701,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
752,1174052600,서울특별시,강동구,상일제2동,1174010300,상일동,20210701,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1589,2811061500,인천광역시,중구,개항동,2811013700,전동,20210701,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1590,2811061500,인천광역시,중구,개항동,2811013800,북성동1가,20210701,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1591,2811061500,인천광역시,중구,개항동,2811013900,북성동2가,20210701,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1592,2811061500,인천광역시,중구,개항동,2811014000,북성동3가,20210701,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1593,2811061500,인천광역시,중구,개항동,2811014100,선린동,20210701,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1594,2811061500,인천광역시,중구,개항동,2811014200,송월동1가,20210701,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1595,2811061500,인천광역시,중구,개항동,2811014300,송월동2가,20210701,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1596,2811061500,인천광역시,중구,개항동,2811014400,송월동3가,20210701,nan,nan,NaN,NaN,NaN,nan,NaN,NaN,nan,nan,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. 건축연면적에 mapping2를 merge


In [104]:
bld_tr.head()

,관리건축물대장PK,건물명,대지면적,건축면적,연면적,용적률산정연면적,용적률,세대수,가구수,도로명주소,지번주소
0,47113-100216867,장량휴먼시아,0.0000,632.9550,"8,409.5540","8,394.6490",0.0000,149,0,경상북도 포항시 북구 장량중앙로 17,경상북도 포항시 북구 양덕동 336-1
1,48220-100186682,미수휴먼시아,0.0000,661.9600,"1,213.8700",240.8300,0.0000,0,0,경상남도 통영시 미우지해안로 107,경상남도 통영시 미수동 695
2,30200-100200744,도안휴먼시아4단지,0.0000,707.1500,"4,636.4100","4,636.4100",0.0000,102,0,대전광역시 유성구 상대로 40,대전광역시 유성구 상대동 465
3,41360-100216608,휴먼시아,0.0000,385.9900,"6,017.7600","5,951.0800",0.0000,76,0,경기도 남양주시 늘을3로 8,경기도 남양주시 호평동 712
4,45140-100190554,장신휴먼시아1단지,0.0000,556.1800,"6,611.9000","6,611.9000",0.0000,84,0,전라북도 익산시 오산면 선화로4길 66,전라북도 익산시 오산면 장신리 1130


In [189]:
juso_tr = list(map(lambda x: x.split()[:-1] , bld_tr.지번주소))
#for idx, j in enumerate (bld_tr.지번주소):
    #jj = j.split()[:-1]
 #   if j.split()[:-1][-1][-1] == "리":
  #      juso_tr[idx] = j.split()[:-1][:-1]
juso_tr = list(map(lambda x: ''.join(x) , juso_tr))
juso_tr

['경상북도포항시북구양덕동',
 '경상남도통영시미수동',
 '대전광역시유성구상대동',
 '경기도남양주시호평동',
 '전라북도익산시오산면장신리',
 '전라북도익산시오산면장신리',
 '경기도성남시분당구백현동',
 '대전광역시유성구지족동',
 '강원도횡성군횡성읍읍하리',
 '광주광역시남구백운동',
 '광주광역시남구백운동',
 '강원도강릉시교동',
 '강원도강릉시포남동',
 '강원도강릉시입암동',
 '경상남도거제시장평동',
 '충청남도당진시석문면통정리',
 '충청남도공주시옥룡동',
 '부산광역시기장군기장읍교리',
 '경상남도김해시구산동',
 '경상남도김해시삼문동',
 '대전광역시서구관저동',
 '대전광역시서구관저동',
 '대전광역시서구월평동',
 '대전광역시중구문화동',
 '대전광역시유성구반석동',
 '대전광역시대덕구법동',
 '대전광역시중구중촌동',
 '경상남도창원시마산회원구내서읍삼계리',
 '경상남도창원시마산회원구내서읍상곡리',
 '경상남도밀양시가곡동',
 '경상남도밀양시삼문동',
 '충청남도보령시명천동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구금곡동',
 '부산광역시북구덕천동',
 '부산광역시북구만덕동',
 '부산광역시사상구모라동',
 '부산광역시해운대구반송동',
 '부산광역시북구금곡동',
 '충청남도부여군부여읍쌍북리',
 '경상남도사천시벌리동',
 '충청남도서산시석림동',
 '강원도속초시교동',
 '울산광역시북구화봉동',
 '강원도원주시문막읍건등리',
 '부산광역시기장군정관읍용수리',
 '제주특별자치도제주시아라일동',
 '제주특별자치도제주시노형동',
 '제주특별자치도제주시화북일동',
 '경상남도진주시가좌동',
 '경상남도진주시평거동',
 '경상남도창원시진해구자은동',
 '경상남도창녕군창녕읍말흘리',
 '충청남도천안시서북구백석동',
 '충청남도천안시서북구성정동',
 '충청남도천안시서북구쌍용동',
 '충청남도청양군청양읍읍내리',
 '강원도춘천시후평동',
 '강원도태백시철암동',
 '경상남도통영시도남동',


In [190]:
bld_tr["juso"] = juso_tr

In [191]:
juso_tst = list(map(lambda x: x.split()[:-1] , bld_tst.지번주소))
#for idx, j in enumerate (bld_tst.지번주소):
    #jj = j.split()[:-1]
 #   if j.split()[:-1][-1][-1] == "리":
  #      juso_tst[idx] = j.split()[:-1][:-1]
juso_tst = list(map(lambda x: ''.join(x) , juso_tst))
juso_tst

['경기도광명시소하동',
 '경기도광명시소하동',
 '전라북도익산시오산면장신리',
 '경기도성남시분당구삼평동',
 '경기도성남시분당구백현동',
 '전라북도군산시미장동',
 '전라북도김제시교동',
 '경상남도남해군남해읍평리',
 '충청남도논산시대교동',
 '대전광역시서구월평동',
 '부산광역시부산진구당감동',
 '부산광역시사상구모라동',
 '부산광역시해운대구좌동',
 '강원도속초시조양동',
 '충청남도아산시읍내동',
 '경상남도양산시상북면대석리',
 '울산광역시남구달동',
 '울산광역시북구호계동',
 '울산광역시북구호계동',
 '울산광역시북구화봉동',
 '강원도원주시명륜동',
 '강원도원주시무실동',
 '강원도원주시무실동',
 '강원도원주시태장동',
 '부산광역시기장군정관읍모전리',
 '제주특별자치도제주시애월읍고성리',
 '제주특별자치도제주시노형동',
 '충청남도천안시서북구쌍용동',
 '강원도춘천시효자동',
 '충청북도제천시강제동',
 '경기도용인시기흥구서천동',
 '경기도용인시기흥구농서동',
 '충청북도음성군삼성면덕정리',
 '전라북도익산시함열읍와리',
 '경상북도경산시상방동',
 '경상북도고령군다산면상곡리',
 '경기도광명시소하동',
 '경기도광명시소하동',
 '광주광역시남구노대동',
 '광주광역시광산구흑석동',
 '광주광역시북구양산동',
 '경상북도구미시구평동',
 '경기도군포시부곡동',
 '전라북도김제시검산동',
 '경기도남양주시진접읍금곡리',
 '대전광역시유성구하기동',
 '충청남도논산시취암동',
 '대구광역시달성군다사읍매곡리',
 '전라남도무안군삼향읍남악리',
 '경기도부천시소사본동',
 '강원도삼척시건지동',
 '강원도삼척시도계읍흥전리',
 '제주특별자치도서귀포시대정읍하모리',
 '부산광역시해운대구송정동',
 '경상남도양산시중부동',
 '경기도양주시만송동',
 '경기도양주시만송동',
 '경기도양주시고암동',
 '제주특별자치도제주시외도일동',
 '경기도용인시기흥구청덕동',
 '경기도용인시기흥구청덕동',
 '경기도용인시수지구죽전동',
 '경기도용인시수지

In [192]:
bld_tst["juso"] = juso_tst

In [193]:
mapping1.시군구명 = mapping1.시군구명.fillna('')
mapping1.읍면동명 = mapping1.읍면동명.fillna('')
mapping1.동리명 = mapping1.동리명.fillna('')
mapping1.행정기관_x = mapping1.행정기관_x.fillna('')

In [197]:
temp = []
for i in range(len(mapping1)):
        
    if mapping1.읍면동명[i]!= '':
        if mapping1.읍면동명[i].strip()[-1] in ["면","읍"]:
            temp.append(mapping1["시도명"][i].strip()+mapping1["시군구명"][i].strip()+mapping1["읍면동명"][i].strip()+mapping1["동리명"][i].strip())
        else:
            temp.append(mapping1["시도명"][i].strip()+mapping1["시군구명"][i].strip()+mapping1["동리명"][i].strip())
    else: 
        temp.append(mapping1["시도명"][i].strip()+mapping1["시군구명"][i].strip()+mapping1["동리명"][i].strip())
     
    
mapping1["주소"] = temp

In [205]:
mapping1.행정기관_x = list(map(lambda x: x.replace(" ",'') , mapping1.행정기관_x))
mapping1.주소 = list(map(lambda x: x.replace(" ",'') , mapping1.주소))

In [210]:
len(set(bld_tr.juso) - set(mapping1.주소))

0

In [209]:
set(bld_tr.juso) - set(mapping1.주소)

set()

In [214]:
len(set(bld_tst.juso) - set(mapping1.주소))

0

In [215]:
mrg_tr = bld_tr.merge(mapping1,left_on = "juso",right_on = "주소",how = "left")
mrg_tr

,관리건축물대장PK,건물명,대지면적,건축면적,연면적,용적률산정연면적,용적률,세대수_x,가구수,도로명주소,지번주소,juso,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자,...,남50~59세,남60~69세,남70~79세,남80~89세,남90~99세,남100세 이상,여 인구수,여연령구간인구수,여0~9세,여10~19세,여20~29세,여30~39세,여40~49세,여50~59세,여60~69세,여70~79세,여80~89세,여90~99세,여100세 이상,주소
0,47113-100216867,장량휴먼시아,0.0000,632.9550,"8,409.5540","8,394.6490",0.0000,149,0,경상북도 포항시 북구 장량중앙로 17,경상북도 포항시 북구 양덕동 336-1,경상북도포항시북구양덕동,4711370000,경상북도,포항시 북구,장량동,4711312200,양덕동,19950101,nan,...,"5,974","4,133","1,247",269,20,0.0000,"36,038","36,038","3,847","4,215","4,084","5,001","6,892","6,151","3,898","1,219",611,117,3,경상북도포항시북구양덕동
1,48220-100186682,미수휴먼시아,0.0000,661.9600,"1,213.8700",240.8300,0.0000,0,0,경상남도 통영시 미우지해안로 107,경상남도 통영시 미수동 695,경상남도통영시미수동,4822066500,경상남도,통영시,미수동,4822011500,미수동,20101231,nan,...,980,935,332,64,5,0.0000,"5,662","5,662",396,645,457,554,942,"1,032",955,435,216,29,1,경상남도통영시미수동
2,30200-100200744,도안휴먼시아4단지,0.0000,707.1500,"4,636.4100","4,636.4100",0.0000,102,0,대전광역시 유성구 상대로 40,대전광역시 유성구 상대동 465,대전광역시유성구상대동,3020061000,대전광역시,유성구,원신흥동,3020011500,상대동,20130916,nan,...,"3,110","2,203",757,221,16,0.0000,"23,897","23,897","2,584","2,495","2,547","4,013","4,584","3,623","2,630",864,440,113,4,대전광역시유성구상대동
3,41360-100216608,휴먼시아,0.0000,385.9900,"6,017.7600","5,951.0800",0.0000,76,0,경기도 남양주시 늘을3로 8,경기도 남양주시 호평동 712,경기도남양주시호평동,4136051000,경기도,남양주시,호평동,4136010100,호평동,19950101,nan,...,"4,256","2,404","1,353",474,45,2.0000,"29,479","29,479","3,039","3,501","3,001","3,994","6,191","4,149","2,917","1,778",750,150,9,경기도남양주시호평동
4,45140-100190554,장신휴먼시아1단지,0.0000,556.1800,"6,611.9000","6,611.9000",0.0000,84,0,전라북도 익산시 오산면 선화로4길 66,전라북도 익산시 오산면 장신리 1130,전라북도익산시오산면장신리,4514031000,전라북도,익산시,오산면,4514031022,장신리,19950510,nan,...,676,733,391,173,19,2.0000,"3,931","3,931",254,304,307,333,478,634,750,479,324,68,0,전라북도익산시오산면장신리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,26290-100212522,용호행복주택,0.0000,0.0000,152.2900,0.0000,0.0000,0,0,부산광역시 남구 동명로 183,부산광역시 남구 용호동 376-9,부산광역시남구용호동,2629058000,부산광역시,남구,용호제2동,2629010700,용호동,19950101,nan,...,"1,284","1,284",754,187,12,1.0000,"8,621","8,621",668,850,714,905,"1,410","1,368","1,498",787,350,66,5,부산광역시남구용호동
457,26290-100212522,용호행복주택,0.0000,0.0000,152.2900,0.0000,0.0000,0,0,부산광역시 남구 동명로 183,부산광역시 남구 용호동 376-9,부산광역시남구용호동,2629059000,부산광역시,남구,용호제3동,2629010700,용호동,19950101,nan,...,"1,007","1,136",735,196,10,0.0000,"6,351","6,351",244,427,628,492,789,"1,093","1,407",861,360,47,3,부산광역시남구용호동
458,26290-100212522,용호행복주택,0.0000,0.0000,152.2900,0.0000,0.0000,0,0,부산광역시 남구 동명로 183,부산광역시 남구 용호동 376-9,부산광역시남구용호동,2629060000,부산광역시,남구,용호제4동,2629010700,용호동,19950101,nan,...,631,713,409,117,15,3.0000,"4,431","4,431",226,331,422,471,644,698,921,489,201,23,5,부산광역시남구용호동
459,43150-100289809,제천 공공실버주택,0.0000,0.0000,88.8600,0.0000,0.0000,0,0,충청북도 제천시 용두천로40길 28,충청북도 제천시 고암동 1273,충청북도제천시고암동,4315051800,충청북도,제천시,의림지동,4315011500,고암동,20170101,nan,...,896,689,263,113,15,1.0000,"5,063","5,063",368,536,602,518,825,891,702,352,226,41,2,충청북도제천시고암동


In [213]:
mrg_tr.isnull().sum()

관리건축물대장PK      0
건물명            0
대지면적           0
건축면적           0
연면적            0
용적률산정연면적       0
용적률            0
세대수_x          0
가구수            0
도로명주소          0
지번주소           0
juso           0
행정동코드          0
시도명            0
시군구명           0
읍면동명           0
법정동코드          0
동리명            0
생성일자           0
말소일자         461
행정기관코드_x       1
행정기관_x         0
총인구수_x         1
세대수_y          1
세대당 인구         1
남자 인구수         1
여자 인구수         1
남여 비율          1
행정기관코드_y       1
행정기관_y         1
총인구수_y         1
총연령구간인구수       1
총0~9세          1
총10~19세        1
총20~29세        1
총30~39세        1
총40~49세        1
총50~59세        1
총60~69세        1
총70~79세        1
총80~89세        1
총90~99세        1
총100세이상        1
남 인구수          1
남연령구간인구수       1
남0~9세          1
남10~19세        1
남20~29세        1
남30~39세        1
남40~49세        1
남50~59세        1
남60~69세        1
남70~79세        1
남80~89세        1
남90~99세        1
남100세 이상       1
여 인구수          1
여연령구간인구수       1
여0~9세         

In [216]:
mrg_tst = bld_tst.merge(mapping1,left_on = "juso",right_on = "주소",how = "left")
mrg_tst

,관리건축물대장PK,건물명,대지면적,건축면적,연면적,용적률산정연면적,용적률,세대수_x,가구수,도로명주소,지번주소,juso,행정동코드,시도명,시군구명,읍면동명,법정동코드,동리명,생성일자,말소일자,...,남50~59세,남60~69세,남70~79세,남80~89세,남90~99세,남100세 이상,여 인구수,여연령구간인구수,여0~9세,여10~19세,여20~29세,여30~39세,여40~49세,여50~59세,여60~69세,여70~79세,여80~89세,여90~99세,여100세 이상,주소
0,41210-100196153,광명역세권 휴먼시아,0,13.3200,13.3200,13.3200,0.0000,0,0,경기도 광명시 서원로 25,경기도 광명시 소하동 1391,경기도광명시소하동,4121064000,경기도,광명시,소하1동,4121010400,소하동,19880423,nan,...,"2,444","1,790",736,193,22,0.0000,"16,312","16,312","1,428","2,075","1,798","2,049","3,152","2,476","1,951",856,439,86,2,경기도광명시소하동
1,41210-100196153,광명역세권 휴먼시아,0,13.3200,13.3200,13.3200,0.0000,0,0,경기도 광명시 서원로 25,경기도 광명시 소하동 1391,경기도광명시소하동,4121065000,경기도,광명시,소하2동,4121010400,소하동,19880423,nan,...,"4,140","2,763","1,130",319,33,2.0000,"25,253","25,253","2,239","2,433","3,322","4,007","4,274","3,978","2,948","1,306",631,112,3,경기도광명시소하동
2,41210-100193203,광명역세권 휴먼시아,0,456.8600,"3,773.7700","3,723.4400",0.0000,64,0,경기도 광명시 성채안로 26,경기도 광명시 소하동 1373,경기도광명시소하동,4121064000,경기도,광명시,소하1동,4121010400,소하동,19880423,nan,...,"2,444","1,790",736,193,22,0.0000,"16,312","16,312","1,428","2,075","1,798","2,049","3,152","2,476","1,951",856,439,86,2,경기도광명시소하동
3,41210-100193203,광명역세권 휴먼시아,0,456.8600,"3,773.7700","3,723.4400",0.0000,64,0,경기도 광명시 성채안로 26,경기도 광명시 소하동 1373,경기도광명시소하동,4121065000,경기도,광명시,소하2동,4121010400,소하동,19880423,nan,...,"4,140","2,763","1,130",319,33,2.0000,"25,253","25,253","2,239","2,433","3,322","4,007","4,274","3,978","2,948","1,306",631,112,3,경기도광명시소하동
4,45140-100188059,장신휴먼시아3단지,0,503.1300,"6,006.8700","5,953.7800",0.0000,84,0,전라북도 익산시 오산면 선화로4길 14,전라북도 익산시 오산면 장신리 1126,전라북도익산시오산면장신리,4514031000,전라북도,익산시,오산면,4514031022,장신리,19950510,nan,...,676,733,391,173,19,2.0000,"3,931","3,931",254,304,307,333,478,634,750,479,324,68,0,전라북도익산시오산면장신리
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,43130-100283541,충주호암 행복주택,0,0.0000,18.7300,0.0000,0.0000,0,0,충청북도 충주시 호암수청1로 30,충청북도 충주시 호암동 1067,충청북도충주시호암동,4313058000,충청북도,충주시,호암.직동,4313010700,호암동,19880423,nan,...,"1,414",899,344,117,3,0.0000,"9,306","9,306",996,"1,179",990,"1,314","1,727","1,449",923,417,258,52,1,충청북도충주시호암동
164,50130-100342705,서귀포성산엘에이치아파트,0,40.9200,40.9200,40.9200,0.0000,0,0,제주특별자치도 서귀포시 성산읍 고성동서로 33,제주특별자치도 서귀포시 성산읍 고성리 1141-1,제주특별자치도서귀포시성산읍고성리,5013025900,제주특별자치도,서귀포시,성산읍,5013025924,고성리,20060701,nan,...,"1,743","1,507",732,308,23,0.0000,"7,373","7,373",431,488,649,662,876,"1,385","1,273",805,644,152,8,제주특별자치도서귀포시성산읍고성리
165,43130-100285480,충주호암1단지,0,211.0500,207.4500,207.4500,0.0000,0,0,충청북도 충주시 호암수청2로 56,충청북도 충주시 호암동 1066,충청북도충주시호암동,4313058000,충청북도,충주시,호암.직동,4313010700,호암동,19880423,nan,...,"1,414",899,344,117,3,0.0000,"9,306","9,306",996,"1,179",990,"1,314","1,727","1,449",923,417,258,52,1,충청북도충주시호암동
166,42790-100205071,화천신읍 공공실버주택,0,34.4400,0.0000,0.0000,0.0000,0,0,강원도 화천군 화천읍 산천어길 196,강원도 화천군 화천읍 신읍리 1818,강원도화천군화천읍신읍리,4279025000,강원도,화천군,화천읍,4279025025,신읍리,19880423,nan,...,695,617,301,145,12,1.0000,"3,995","3,995",331,368,454,439,523,594,532,421,290,40,3,강원도화천군화천읍신읍리


In [217]:
mrg_tst.isnull().sum()

관리건축물대장PK      0
건물명            0
대지면적           0
건축면적           0
연면적            0
용적률산정연면적       0
용적률            0
세대수_x          0
가구수            0
도로명주소          0
지번주소           0
juso           0
행정동코드          0
시도명            0
시군구명           0
읍면동명           0
법정동코드          0
동리명            0
생성일자           0
말소일자         168
행정기관코드_x       1
행정기관_x         0
총인구수_x         1
세대수_y          1
세대당 인구         1
남자 인구수         1
여자 인구수         1
남여 비율          1
행정기관코드_y       1
행정기관_y         1
총인구수_y         1
총연령구간인구수       1
총0~9세          1
총10~19세        1
총20~29세        1
총30~39세        1
총40~49세        1
총50~59세        1
총60~69세        1
총70~79세        1
총80~89세        1
총90~99세        1
총100세이상        1
남 인구수          1
남연령구간인구수       1
남0~9세          1
남10~19세        1
남20~29세        1
남30~39세        1
남40~49세        1
남50~59세        1
남60~69세        1
남70~79세        1
남80~89세        1
남90~99세        1
남100세 이상       1
여 인구수          1
여연령구간인구수       1
여0~9세         

mrg1.to_csv("/Users/seungji/Desktop/Dacon/mrg1.csv",index = False)